# VIDEO CORE ANALYSIS SYSTEM

## Setup: Clone Repository


In [ ]:
!git clone https://github.com/fuadnafiz98/video-core.git
%cd video-core

### Install Dependencies

In [ ]:
!pip install opencv-contrib-python --upgrade
!pip install scenedetect[opencv] click questionary numpy

### Check GPU Availability

In [ ]:
import cv2
import sys

print("=" * 50)
print("HARDWARE CONFIGURATION")
print("=" * 50)

# Check OpenCV CUDA support
cuda_available = cv2.cuda.getCudaEnabledDeviceCount() > 0

if cuda_available:
    print(f"✓ GPU ENABLED")
    print(f"  Devices: {cv2.cuda.getCudaEnabledDeviceCount()}")
    print(f"  Device: {cv2.cuda.getDevice()}")

    # Try to get GPU info
    try:
        import subprocess
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                              capture_output=True, text=True)
        if result.returncode == 0:
            gpu_info = result.stdout.strip().split(',')
            print(f"  Name: {gpu_info[0].strip()}")
            print(f"  Memory: {gpu_info[1].strip()}")
    except:
        pass

    print(f"\n  Processing Mode: GPU Accelerated")
else:
    print(f"✗ GPU NOT AVAILABLE")
    print(f"  Processing Mode: CPU Only")

print(f"\nOpenCV Version: {cv2.__version__}")
print(f"Python Version: {sys.version.split()[0]}")
print("=" * 50)

### Upload Video

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

print("Upload your video file:")
uploaded = files.upload()

video_dir = Path("videos")
video_dir.mkdir(exist_ok=True)

for filename in uploaded.keys():
    shutil.move(filename, video_dir / filename)
    print(f"Moved {filename} to videos/")

### Run Analysis

In [ ]:
from src.extractors.shot_cut_detector import ShotCutDetector
from pathlib import Path
import json
import time

videos = [f for f in Path("videos").glob("*.*") if f.suffix.lower() in ['.mp4', '.avi', '.mov', '.mkv', '.webm']]

if not videos:
    print("No videos found. Please upload a video first.")
else:
    video_path = videos[0]
    print(f"\nProcessing: {video_path.name}\n")

    start_time = time.time()

    detector = ShotCutDetector(threshold=27.0, min_scene_len=15, use_gpu=True)
    shot_result = detector.extract(str(video_path))

    elapsed_time = time.time() - start_time

    output_data = {
        "video_file": str(video_path),
        "processing_time_seconds": round(elapsed_time, 2),
        "features": {
            "shot_cuts": shot_result
        }
    }

    results_dir = Path("results") / video_path.stem
    results_dir.mkdir(parents=True, exist_ok=True)
    output_file = results_dir / "output.json"

    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=2)

    print("\n" + "=" * 50)
    print("ANALYSIS COMPLETE")
    print("=" * 50)
    print(f"  PROCESSING TIME: {elapsed_time:.2f}s")
    print(f"  CUTS DETECTED: {shot_result['total_cuts']}")
    print(f"  AVG SCENE: {shot_result['avg_scene_length']}s")
    print(f"  DURATION: {shot_result['duration']}s")
    print(f"  MODE: {shot_result.get('processing_mode', 'N/A')}")
    print(f"\nResults saved to: {output_file}")
    print("=" * 50)

### Performance Comparison (Optional)
Run this cell to compare GPU vs CPU performance on motion analysis.

In [ ]:
from src.extractors.motion_analyzer import MotionAnalyzer
from pathlib import Path
import time
import cv2

videos = [f for f in Path("videos").glob("*.*") if f.suffix.lower() in ['.mp4', '.avi', '.mov', '.mkv', '.webm']]

if not videos:
    print("No videos found.")
elif cv2.cuda.getCudaEnabledDeviceCount() == 0:
    print("GPU not available. Skipping comparison.")
else:
    video_path = videos[0]
    print(f"Benchmarking: {video_path.name}\n")

    # GPU Test
    print("Testing GPU mode...")
    gpu_analyzer = MotionAnalyzer(sample_rate=5, downscale=2, show_progress=False, use_gpu=True)
    gpu_start = time.time()
    gpu_result = gpu_analyzer.extract(str(video_path))
    gpu_time = time.time() - gpu_start

    # CPU Test
    print("Testing CPU mode...")
    cpu_analyzer = MotionAnalyzer(sample_rate=5, downscale=2, show_progress=False, use_gpu=False)
    cpu_start = time.time()
    cpu_result = cpu_analyzer.extract(str(video_path))
    cpu_time = time.time() - cpu_start

    # Results
    speedup = cpu_time / gpu_time if gpu_time > 0 else 0

    print("\n" + "=" * 50)
    print("PERFORMANCE COMPARISON")
    print("=" * 50)
    print(f"GPU Time: {gpu_time:.2f}s")
    print(f"CPU Time: {cpu_time:.2f}s")
    print(f"Speedup: {speedup:.2f}x faster")
    print("=" * 50)

### View Results

In [ ]:
import json
from pathlib import Path

result_files = list(Path("results").rglob("output.json"))

if result_files:
    with open(result_files[0]) as f:
        data = json.load(f)
    print(json.dumps(data, indent=2))
else:
    print("No results found.")

### Download Results

In [ ]:
from google.colab import files
from pathlib import Path

result_files = list(Path("results").rglob("output.json"))

for result_file in result_files:
    files.download(str(result_file))